In [2]:
from dotenv import load_dotenv
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from typing import Any
from pathlib import Path

# Load environment variables from .env file
load_dotenv()


try:
    credential = DefaultAzureCredential(exclude_environment_credential=True, exclude_shared_token_cache_credential=True, exclude_managed_identity_credential=True)
except Exception as ex:
    # Fall back to InteractiveBrowserCredential if DefaultAzureCredential fails
    credential = InteractiveBrowserCredential()

# Create the project client using the connection string
project_client = AIProjectClient(
    endpoint=os.environ["ENDPOINT"],
    subscription_id=os.environ["SUBSCRIPTION_ID"],
    resource_group_name=os.environ["RESOURCE_GROUP_NAME"],
    project_name=os.environ["PROJECT_NAME"],
    credential=credential,
)

# Print the project client details
print(f"Endpoint: {project_client._config3.endpoint}")
print(f"Subscription ID: {project_client._config3.subscription_id}")
print(f"Resource Group Name: {project_client._config3.resource_group_name}")
print(f"Project Name: {project_client._config3.project_name}")
print(f"Credential: {project_client._config3.credential}")

with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="my-assistant",
        instructions="You are helpful assistant",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

    # Get the last message from the sender
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    # Generate an image file for the bar chart
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")

    # Print the file path(s) from the messages
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")
        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Endpoint: https://eastus.api.azureml.ms
Subscription ID: 1278a874-89fc-418c-b6b9-ac763b000415
Resource Group Name: aiagents-demo
Project Name: ai-standard-project-hz7g
Credential: <azure.identity._credentials.default.DefaultAzureCredential object at 0x000001F502963C10>
Created agent, agent ID: asst_2sv6ayrJsfqNtkCZXkmZtgAm
Created thread, thread ID: thread_1y0oFZBeW4S0aq5KuDFRKWEI
Created message, message ID: msg_dIxM5HwbpNWOLuCewG3OmuHw
Run finished with status: RunStatus.COMPLETED
Messages: <azure.ai.projects.models._patch.ThreadMessages object at 0x000001F50293FD10>
Last Message: Here is the bar chart displaying the operating profit of the companies. You can download it using the following link:

[Download the bar chart](sandbox:/mnt/data/operating_profit_chart.png)
Image File ID: assistant-5Razye5qCPGlKQ7ZItOtbXe6
Saved image file to: d:\azure-ai-agents\samples\assistant-5Razye5qCPGlKQ7ZItOtbXe6_image_file.png
File Paths:
Type: file_path
Text: sandbox:/mnt/data/operating_profit_cha